 # Opis notatnika
 W poprzednich krokach pobraliśmy dane oraz przygotowaliśmy bazę `Postgres` na import. Głównym celem w tym notatniku jest  odpowiednie dostosowanie struktury danych z plików źródłowych do formatu zgodnego z `Postgres`, a następnie wgranie ich na nasz serwer. Dzięki temu w późniejszych krokach możemy niezależnie użyć danych do analizy czy raportowania.
 
 Ponownie wcielasz się w rolę Data Engineera, którego zadaniem jest zasilenie bay danych pobranymi danymi. Bez poprawnego załadowania danych nie będziesz w stanie dokonać analizy eksploracyjnej, która jest jednym z wymagań dostarczonych przez klienta.

 Przy wykonywaniu tego notebooka przydadzą się poniższe elementy kursu oraz materiały dodatkowe:
 * `SQL - analiza danych > Zjazd 1 - materiały dodatkowe > Export danych z DB > Python` - w celu użycia połączenia razem z `Pandas`,
 * https://docs.sqlalchemy.org/en/14/core/engines.html - w celu uzupełnienia konfiguracji `Pandas` do `PostgerSQL`,
 * https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html - eksport danych z `Pandas` na bazę danych.

 > Uwaga: Ze względu na wolumen danych zawarty w pliku `flight.csv`, wykonanie tego notatnika może zająć nawet kilkadziesiąt minut lub więcej!

## Połączenie z bazą danych
Tutaj uzupełnij konfigurację połączenia tworząc zmienne takie jak:
- `username` - nazwa użytkownika bazy,
- `password` - hasło do bazy,
- `host` - adres naszej bazy danych, jeśli baza jest postawiona na naszej maszynie wtedy będzie to po prostu `localhost`,
- `database` - nazwa bazy danych np. `postgresql`
- `port` - domyślnie `5432`

> Przetrzymywanie hasła w ten sposób nie jest bezpieczne, co było zaznaczane w trakcie kursu. Lepszym sposobem jest używanie zmiennych środowiskowych, ale na nasze potrzeby nie jest to potrzebne. Dla osób chcących zapoznać się z taką formą zalecamy ten artykuł - [klik](https://developer.vonage.com/blog/21/10/01/python-environment-variables-a-primer).

In [9]:
username = 'postgres'
password = 'SQLisFun'

host = 'localhost'
database = 'airlines'
port = None

 Z pomocą artykułu [click](https://docs.sqlalchemy.org/en/14/core/engines.html) stwórz zmienne `url` oraz `engine`. Zgodnie z dokumentacją `Pandas`, zmienna `engine` będzie potrzebna, by móc wyeksportować dane na serwer `SQL`.

W tym miejscu stwórz zmienne `url` oraz `engine`
> Wskazówka: Zmienna `url` powinna być stworzona zgodnie ze schematem jak we wcześniej podanym artykule, jednak powinna używać zmiennych zdefiniowanych wyżej.

In [10]:
import urllib.parse
from sqlalchemy.engine import URL
from sqlalchemy import create_engine
from sqlalchemy import text
import pandas as pd

In [11]:
url= URL.create(
    "postgresql",
    username=username,
    password=password, 
    host=host,
    database=database,
)

engine = create_engine(url)

 # Załadowanie ramek do obszaru roboczego
 Uzupełnij implementacje funkcji `load_raw_data`, która przyjmuje jeden parametr:
 * `file_name` - nazwa pliku do zaczytania
 Jej zadaniem jest wczytanie surowego pliku, zmodyfikowanie nazw kolumn z `NAZWA_KOLUMNY` na `nazwa_kolumny` oraz zwrócenie tak zmodyfikowanej ramki danych

 Mogą się przydać poniższe element kursu:
 - `Python-Analiza danych -> Dzień 5 - Pandas -> Obróbka danych - częsć 1`
 - `Python-Analiza danych -> Przygotowanie do zjazdu 3 -> Wstęp do Pandas -> Wczytywanie danych do Pandas` - jakie kodowanie mają pliki?

In [12]:
def load_raw_data(file_name):
    return file_name.rename(columns = str.lower)

 # Zaczytanie poszczególnych plików do ramek

 W tym miejscu zaczytaj poszczególne pliki do ramek

In [54]:
aircraft_df = pd.read_csv(
                    r'..\data\raw\aircraft.csv', 
                    sep=',',  
                    decimal='.' 
)

aircraft = load_raw_data(aircraft_df)
aircraft

,manufacture_year,tail_num,number_of_seats
0,1944,N54514,0.0
1,1945,N1651M,0.0
2,1953,N100CE,0.0
3,1953,N141FL,0.0
4,1953,N151FL,0.0
...,...,...,...
7378,2019,N14011,337.0
7379,2019,N16008,337.0
7380,2019,N16009,337.0
7381,2019,N2250U,276.0


In [55]:
airport_list_df = pd.read_csv(
                    r'..\data\raw\airport_list.csv', 
                    sep=',',  
                    decimal='.' 
)

airport_listload_raw_data(airport_list_df)

,origin_airport_id,display_airport_name,origin_city_name,name
0,11638,Fresno Air Terminal,"Fresno, CA","FRESNO YOSEMITE INTERNATIONAL, CA US"
1,13342,General Mitchell Field,"Milwaukee, WI","MILWAUKEE MITCHELL AIRPORT, WI US"
2,13244,Memphis International,"Memphis, TN","MEMPHIS INTERNATIONAL AIRPORT, TN US"
3,15096,Syracuse Hancock International,"Syracuse, NY","SYRACUSE HANCOCK INTERNATIONAL AIRPORT, NY US"
4,10397,Atlanta Municipal,"Atlanta, GA",ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...
...,...,...,...,...
92,13198,Kansas City International,"Kansas City, MO","KANSAS CITY INTERNATIONAL AIRPORT, MO US"
93,10423,Austin - Bergstrom International,"Austin, TX","AUSTIN BERGSTROM INTERNATIONAL AIRPORT, TX US"
94,15370,Tulsa International,"Tulsa, OK","OKLAHOMA CITY WILL ROGERS WORLD AIRPORT, OK US"
95,13303,Miami International,"Miami, FL","MIAMI INTERNATIONAL AIRPORT, FL US"


In [79]:
weather_df = pd.read_csv(
                    r'..\data\raw\airport_weather.csv', 
                    sep=',',  
                    decimal='.' 
)

weather = load_raw_data(weather_df)
weather

,wt18,station,name,date,awnd,prcp,snow,snwd,tavg,tmax,...,pgtm,wt10,wesd,sn32,sx32,psun,tsun,tobs,wt07,wt11
0,NaN,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,2019-01-01,4.70,0.14,0.0,0.0,64.0,66.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,2019-01-02,4.92,0.57,0.0,0.0,56.0,59.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,2019-01-03,5.37,0.15,0.0,0.0,52.0,55.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,2019-01-04,12.08,1.44,0.0,0.0,56.0,66.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,2019-01-05,13.42,0.00,0.0,0.0,49.0,59.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46221,NaN,USW00014762,"PITTSBURGH ALLEGHENY CO AIRPORT, PA US",2020-03-27,3.58,0.21,NaN,NaN,NaN,59.0,...,146.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46222,NaN,USW00014762,"PITTSBURGH ALLEGHENY CO AIRPORT, PA US",2020-03-28,6.93,1.29,NaN,NaN,NaN,77.0,...,1535.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46223,NaN,USW00014762,"PITTSBURGH ALLEGHENY CO AIRPORT, PA US",2020-03-29,16.55,0.02,NaN,NaN,NaN,78.0,...,1408.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46224,NaN,USW00014762,"PITTSBURGH ALLEGHENY CO AIRPORT, PA US",2020-03-30,13.42,0.00,NaN,NaN,NaN,57.0,...,817.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
flight_df = pd.read_csv(
                    r'..\data\raw\flight.csv', 
                    sep=',',  
                    decimal='.' 
)

flight = load_raw_data(flight_df)
flight

,month,day_of_month,day_of_week,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,dep_time,...,crs_elapsed_time,actual_elapsed_time,distance,distance_group,year,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,1,20,7,WN,N204WN,682,10397,11292,605,602.0,...,205,204.0,1199,5,2019,NaN,NaN,NaN,NaN,NaN
1,1,20,7,WN,N8682B,2622,10397,11292,2120,2114.0,...,210,205.0,1199,5,2019,NaN,NaN,NaN,NaN,NaN
2,1,20,7,WN,N717SA,2939,10397,11292,1800,1807.0,...,210,220.0,1199,5,2019,4.0,0.0,10.0,0.0,3.0
3,1,20,7,WN,N709SW,3848,10397,11292,1355,1354.0,...,205,204.0,1199,5,2019,NaN,NaN,NaN,NaN,NaN
4,1,20,7,WN,N7864B,1352,10397,11697,1125,1125.0,...,120,124.0,581,3,2019,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1386115,3,26,4,DL,N350DN,1982,13303,12953,1956,1944.0,...,183,169.0,1096,5,2020,NaN,NaN,NaN,NaN,NaN
1386116,3,26,4,DL,N908DE,1987,13303,10397,1120,1117.0,...,121,109.0,594,3,2020,NaN,NaN,NaN,NaN,NaN
1386117,3,26,4,DL,NaN,1998,13303,10397,1817,NaN,...,125,NaN,594,3,2020,NaN,NaN,NaN,NaN,NaN
1386118,3,26,4,DL,N352NW,2025,13303,10397,1937,1928.0,...,123,107.0,594,3,2020,NaN,NaN,NaN,NaN,NaN


 # Eksport danych na bazę
 Zapoznaj się z dokumentacją metody `Pandas` - [to_sql](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html), której zadaniem jest wyeksportowanie ramki na bazę danych.
 Zwróć szczególną uwagę na poniższe parametry:
 * `if_exists` - jak ma się zachować metoda, gdy ładuje dane na bazę,
 * `con` - połączenie do bazy,
 * `name` - nazwa tabeli, do której ramka ma zostać wgrana,
 * `index` - czy dodawać index z ramki jako kolumnę,
 * `chunksize` - maksymalna liczba wierszy wgrywana za jednym razem.

 > **Uwaga:** 
 > Przed eksportem upewnij się, że tabela jest pusta. Zwróć uwagę na pewną subtelną różnicę pomiędzy wyglądem ramki oraz tabeli docelowej na bazie danych.

Następnie uzupełnij implementację metody `export_table_to_db`, która przyjmuje dwa argumenty:
 * `df` - ramka danych do eksportu,
 * `table_name` - nazwa ramki na bazie.

Zalecamy, aby dodać do metody informację, która ramka jest aktualnie ładowana np.:
 `Loading data into {table_name}...`
 > Ze względu na rozmiar ramki `flight_df`, proces ten może potrwać nawet kilkadziesiąt minut! Z tego względu, na potrzeby testów, zalecamy przekazanie do procedury `export_table_to_db` np. pierwszych 5 wierszy, aby sprawdzić, czy działa, a potem wgrać cały zestaw danych - pamiętając o upszednim usunięciu tamtych.

In [14]:
def export_table_to_db(df, table_name):
    return df.to_sql(name=table_name, con=engine, if_exists = 'append', index = id)

 ## Wgrywanie danych

 ### Wgranie `aircraft_df` do tabeli `aircraft`

In [70]:
export_table_to_db(aircraft,'aircraft')

383

 ### Wgranie `airport_weather_df` do tabeli `airport_weather`

In [71]:
export_table_to_db(weather,'airport_weather')

226

 ### Wgranie `flight_df` do tabeli `flight`
 > Wykonanie tej komórki może zająć kilka-kilknaście minut za względu na ilość danych w ramce.

In [15]:
export_table_to_db(flight,'flight')

120

 ### Wgranie `airport_list_df` do tabeli `airport_list`

In [72]:
export_table_to_db(airport_list,'airport_list')

97

 # Sprawdzenie poprawności wykonania notatnika
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [16]:
def test_data_export(table_name, expected_count, expected_schema):
    real_count = pd.read_sql(f"SELECT COUNT(*) as cnt FROM {table_name}", engine).iloc[0][0]
    
    real_schema = pd.read_sql(f"SELECT * FROM {table_name} LIMIT 0", engine)
    real_schema = set(real_schema.columns)

    expected_schema = set(expected_schema)

    diff = real_schema.symmetric_difference(expected_schema)

    assert len(diff) == 0, ('Nie zgadzają się kolumny tabel....'
    f'\tOczekiwano: {expected_schema}'
    f'\tOtrzymano: {real_schema}'
    f'\tRóżnica: {diff}')

    assert expected_count == real_count, \
        f'Nie zgadza się liczba wierszy, oczekiwano {expected_count}, otrzymano {real_count} - sprawdź, czy nie dane nie zostały wgrane do tabeli "{table_name}" więcej niż raz.'

 ## Sprawdzenie tabeli `aircraft`
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [17]:
aircraft_expected_count = 7383
aircraft_expected_schema = ['id', 'manufacture_year', 'tail_num', 'number_of_seats']

test_data_export('aircraft', aircraft_expected_count, aircraft_expected_schema)

 ## Sprawdzenie tabeli `airport_weather`
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [18]:
airport_weather_expected_count = 46226
airport_weather_expected_schema = [
       'id', 'station', 'name', 'date', 'awnd', 'prcp', 'snow', 'snwd', 'tavg', 
       'tmax', 'tmin', 'wdf2', 'wdf5', 'wsf2', 'wsf5', 'wt01', 'wt08', 'wt02',
       'wt03', 'wt04', 'wt09', 'wt06', 'wt05', 'pgtm', 'wt10', 'wesd', 'sn32',
       'sx32', 'psun', 'tsun', 'tobs', 'wt07', 'wt11', 'wt18']

test_data_export('airport_weather', airport_weather_expected_count, airport_weather_expected_schema)

 ## Sprawdzenie tabeli `flight`
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [19]:
flight_expected_count = 1386120
flight_expected_schema = [
       'id', 'month', 'day_of_month', 'day_of_week', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'dest_airport_id',
       'crs_dep_time', 'dep_time', 'dep_delay_new', 'dep_time_blk',
       'crs_arr_time', 'arr_time', 'arr_delay_new', 'arr_time_blk',
       'cancelled', 'crs_elapsed_time', 'actual_elapsed_time', 'distance',
       'distance_group', 'year', 'carrier_delay', 'weather_delay', 'nas_delay',
       'security_delay', 'late_aircraft_delay']

test_data_export('flight', flight_expected_count, flight_expected_schema)

 ## Sprawdzenie tabeli `airport_list`
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [20]:
airport_list_expected_count = 97
airport_list_expected_schema = ['id', 'origin_airport_id', 'display_airport_name', 'origin_city_name', 'name']

test_data_export('airport_list', airport_list_expected_count, airport_list_expected_schema)


In [21]:
msg = "Wszystko wygląda OK :) Możesz przejść do kolejnego kroku."
print(msg)

Wszystko wygląda OK :) Możesz przejść do kolejnego kroku.


 # Podsumowanie
 W tym notatniku załadowaliśmy pobrane wcześniej pliki na bazę danych. Dzięki temu stworzyliśmy centralne miejsce ich magazynowania, co wykorzystamy zarówno przy analizie danych, jak i przy późniejszej budowie systemu raportowego.